In [0]:
dbutils.widgets.text('p_project_name', "")

In [0]:
v_project_name = dbutils.widgets.get('p_project_name')

In [0]:
table_name= []

for i in dbutils.fs.ls(f'/mnt/kevee/bronze/{v_project_name}/'):
     _name = (i.name.split('/')[0])
     table_name.append(_name.split('.')[0])

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType
import re

In [0]:
# Define a function to replace invalid characters in column names
def replace_col_names(df):
    """
    Replace invalid characters in column names with alphanumeric characters
    and underscores.
    """
    new_col_names = []
    for _old_col_name in df.columns:
        old_col_name =_old_col_name.split('.')[-1]
        new_col_name = re.sub('[ ,;{}()\n\t=]+', '_', old_col_name)
        new_col_names.append(new_col_name)
    # Rename DataFrame with new column names
    return df.toDF(*new_col_names)

In [0]:
# Read data from Parquet files and replace invalid column names
for table in table_name:
    path = f'/mnt/kevee/bronze/{v_project_name}/{table}.parquet'
    schema = spark.read.format('parquet').load(path).schema
    df = spark.read.format('parquet').schema(schema).load(path)

    # Replace invalid characters in column names
    df = replace_col_names(df)
    
    # Write DataFrame to Delta Lake and replace existing data
    output_path = f'/mnt/kevee/silver/{v_project_name}/{table}/'
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
dbutils.notebook.exit('Success')